In [1]:
import numpy as np

from PIL import Image

In [2]:
img_name = 'filter/filter_01.jpg'

img_filter = Image.open(img_name)

filter_arr = np.array(img_filter, dtype=float)

In [3]:
filter_arr = np.array(filter_arr, dtype=np.uint8)

filter_arr = np.transpose(filter_arr, (2, 1, 0))

In [4]:
_, H, W = filter_arr.shape

fft_filter = np.array([np.empty((H, W))] * 3, dtype=complex)

for channel in range(3):
    fft_filter_channel = np.fft.fft2(filter_arr[channel])

    fft_filter[channel] = np.fft.fftshift(fft_filter_channel)

# CPU Image Processing

In [5]:
def normalize_array(array: np.array):
    magnitude_norm = 255 * (array / np.max(array))

    return magnitude_norm.astype(np.uint8)


def process_image_cpu(array: np.array):
    fft_img = np.empty_like(array, dtype=complex)

    for  channel in range(3):
        fft_array = np.fft.fft2(array[channel])

        fft_array = np.fft.fftshift(fft_array)

        fft_img[channel] = fft_array + fft_filter[channel]

        f = np.fft.ifftshift(fft_img[channel])

        fft_img[channel] = np.fft.ifft2(f).real

    return normalize_array(fft_img)

# GPU Image Processing

In [6]:
def process_image_gpu(array: np.array):
    pass

# Webcam

In [7]:
import cv2 as cv

camera = cv.VideoCapture(0)

camera.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
camera.set(cv.CAP_PROP_FRAME_HEIGHT, 720)

if not camera.isOpened():
    print("Error opening camera.")
    exit()

while True:
    status, frame = camera.read()

    try:
        array = np.array(frame, dtype=np.uint8)

        array = np.transpose(array, (2, 1, 0))

        array = process_image_cpu(array)
        #array = process_image_gpu(array)

        array = np.transpose(array, (2, 1, 0))
    except Exception as e:
        continue

    if not status or cv.waitKey(1) & 0xff == ord('q'):
        break

    cv.imshow("Camera", array)

camera.release()

cv.destroyAllWindows()

C:\Users\tiago\AppData\Local\Temp\ipykernel_32988\246209646.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  return magnitude_norm.astype(np.uint8)


KeyboardInterrupt: 